## Load Requirements

In [17]:
import numpy as np


## SVD for a Noisy Matrix


#### Matrix Generation

In [18]:
# Generate the matrix

N,M = 100,50

# Form 2 random vectors
u=np.random.rand(N,1)
v=np.random.rand(M,1)

# Compute the rank-1 matrix using outer product
R_1 = u @ v.T

# Calculate the Frobenius norm of this matrix
R_f = np.linalg.norm(R_1, 'fro')

noise_variance = 0.01 * R_f

noise = np.random.normal(0, np.sqrt(noise_variance), (N,M)) 

R1 = R_1 + noise

# Display the shape of the matrix

print("The matrix Shape:", R1.shape)


The matrix Shape: (100, 50)


#### SVD Decomposition

In [19]:
# Decompose the matrix using SVD

U1, s1, V1 = np.linalg.svd(R1, full_matrices=False)

print("U Shape:", U1.shape)

print("s Shape:", s1.shape)

print("V Shape:", V1.shape)


U Shape: (100, 50)
s Shape: (50,)
V Shape: (50, 50)


#### Matrix Reconstruction

In [20]:
# Reconstruct the matrix R using only the first singular value and the corresponidng singular vectors

R_reconstructed = s1[0] * np.outer(U1[:,0], V1[0,:])

#### Analysis

In [21]:
# output the Reconstructed values of U and V

U_reconstructed, s_constructed, V_reconstructed = np.linalg.svd(R_reconstructed, full_matrices=False)

print("Reconstructed U:", U_reconstructed)

print("Reconstructed V:", V_reconstructed)

Reconstructed U: [[-1.40545577e-01 -3.88810651e-01 -1.13780532e-02 ...  3.40681904e-02
  -1.43403874e-02 -9.00852545e-01]
 [-1.69706563e-01 -2.21097275e-01  5.92597076e-02 ...  3.37317189e-02
   7.60568510e-02  1.30611266e-01]
 [-2.67237109e-02  2.29133000e-02 -5.73232923e-03 ...  3.63035398e-02
  -1.25668939e-01 -2.06068320e-03]
 ...
 [-8.87169709e-03 -7.87872014e-03  2.77197068e-04 ... -1.93479828e-02
  -5.45768519e-02  3.07122692e-03]
 [-1.36255590e-01 -1.35885062e-01  5.75929384e-02 ... -1.01260210e-01
  -1.04762409e-01  8.13625055e-02]
 [-2.92055852e-02  1.74626444e-02  3.86346975e-02 ... -1.39546579e-01
   5.08470049e-02 -4.19144948e-03]]
Reconstructed V: [[-7.96681771e-03 -5.91892935e-02 -1.10278124e-01 ... -1.01730542e-01
  -1.19060360e-01 -3.74743370e-02]
 [ 0.00000000e+00 -3.78180374e-02 -1.73965432e-01 ... -4.58704945e-03
   1.16330169e-02 -3.12785770e-03]
 [ 0.00000000e+00  2.08108812e-02 -4.63045880e-03 ...  1.03924411e-01
  -5.73311891e-02  5.39339566e-02]
 ...
 [ 0.00000

In [22]:
# Computer the Root Mean Squared Error between the original matrix R and the reconstructed matrix R_reconstructed 

RMSE = np.sqrt(np.mean((R1 - R_reconstructed)**2))

print("RMSE:", RMSE)

# Computer the Root Mean Squared Errors of U and V

RMSE_U = np.sqrt(np.mean((U1 - U_reconstructed)**2))

RMSE_V = np.sqrt(np.mean((V1- V_reconstructed)**2))

print("RMSE_U:", RMSE_U)
print("RMSE_V:", RMSE_V)

RMSE: 0.49264272196866243
RMSE_U: 0.14022221684691455
RMSE_V: 0.19550942317387476


**The impact of noise:** : 

1. The RMSE between the original and reconstructed R suggests that noise increases the overall discrepancy between the original noisy matrix and the reconstructed matrix. It impacts the overall rank of the matrix so just using the first singular value to construct matrix would lose some imformation of the matrix.
2. The RMSE between the original and reconstructed U and V suggests that noise increases the overall discrepancy between the original and the reconstructed singular vectors. For the rank-1 matrix, ideally only the first singular value and corresponding singular vectors would be non-zero. However, after noising, additional vectors may become non-zero but those corrisponding singular vectors of reconstructed matrix are not zero, which impacts the overall discrepancy between the original and reconstructed singular vectors.

## Matrix Factorization of an Imcomplete Matrix

#### Matrix Generation

In [24]:
## Generate the Matrix 

# Calculate the total number of elements and 30% of that
total_elements = N * M
num_missing = int(0.3 * total_elements)

# Randomly select indices to set as missing
missing_indices = np.unravel_index(
    np.random.choice(total_elements, num_missing, replace=False), (N, M)
)

# Create a copy of the original matrix and set the missing values to nan

R_missing = np.copy(R_1)

R_missing[missing_indices] = np.nan



#### Matrix Factorization

In [26]:
# SGD for matrix factorization

def SGD_factorization(learning_rate, regularization, num_epochs, R_missing, output, Rank):

    K= Rank
    U_factorized = np.random.rand(N,K)
    V_factorized = np.random.rand(M,K) # initialze the U and V matrices

    for epoch in range(num_epochs):

        for i in range(N):
            for j in range(M):
                
                # Only consider the observed values
                if not np.isnan(R_missing[i,j]):

                    prediction = np.dot(U_factorized[i,:], V_factorized[j,:])
                    error = R_missing[i,j] - prediction

                    # Update the U and V matrices
                    U_factorized[i,:] += learning_rate * (error * V_factorized[j,:] - regularization * U_factorized[i,:])
                    V_factorized[j,:] += learning_rate * (error * U_factorized[i,:] - regularization * V_factorized[j,:]) 
                    
        if output:
            if epoch % 10 == 0:
            
                # Calculatre the total loss on observed entrices
                observed_indices = ~np.isnan(R_missing)
                loss = np.sum((R_missing[observed_indices] - (U_factorized @ V_factorized.T)[observed_indices])**2)
                print("Epoch:", epoch, "Loss:", loss)
    
    return U_factorized, V_factorized

U_factorized,V_factorized = SGD_factorization(0.01, 0, 200, R_missing,True,1)

Epoch: 0 Loss: 262.23028902885807
Epoch: 10 Loss: 20.437717632531182
Epoch: 20 Loss: 1.3836255332651866
Epoch: 30 Loss: 0.11393311358811087
Epoch: 40 Loss: 0.010625924773048858
Epoch: 50 Loss: 0.0010949112046464199
Epoch: 60 Loss: 0.00012265331457071626
Epoch: 70 Loss: 1.4733668957722918e-05
Epoch: 80 Loss: 1.8752354329138669e-06
Epoch: 90 Loss: 2.502176701418193e-07
Epoch: 100 Loss: 3.46798422223587e-08
Epoch: 110 Loss: 4.953206359071287e-09
Epoch: 120 Loss: 7.242185822727976e-10
Epoch: 130 Loss: 1.0781798835276882e-10
Epoch: 140 Loss: 1.6274159289547836e-11
Epoch: 150 Loss: 2.4822703543529292e-12
Epoch: 160 Loss: 3.8162285999731485e-13
Epoch: 170 Loss: 5.902155713602591e-14
Epoch: 180 Loss: 9.169367259399356e-15
Epoch: 190 Loss: 1.4293512867112417e-15


#### Missing Data Imputation

In [27]:
# Use the factorized matrices U and V to reconstruct the matrix R

R_reconstructed_missing = U_factorized @ V_factorized.T



#### Analysis

In [28]:
# Output the reconstructed values of U and V

print(U_factorized,V_factorized.T)

[[0.7152564 ]
 [0.81904694]
 [0.04719282]
 [0.96831662]
 [0.17734362]
 [0.85194888]
 [0.13261678]
 [0.04605048]
 [0.87004177]
 [0.61871822]
 [0.00150608]
 [0.11084721]
 [0.82864751]
 [0.85033771]
 [0.36328974]
 [0.95287898]
 [0.78230286]
 [0.16678854]
 [0.5565714 ]
 [0.30847425]
 [0.19747275]
 [0.35203172]
 [0.80234439]
 [0.37382909]
 [0.87357362]
 [0.63027812]
 [0.80451471]
 [0.88970243]
 [0.08598488]
 [0.76837801]
 [0.44208659]
 [0.13032667]
 [0.05207936]
 [0.68158423]
 [0.75143342]
 [0.75007879]
 [0.02469512]
 [0.65278551]
 [0.34505139]
 [0.29632159]
 [0.32665468]
 [0.40141623]
 [0.6508986 ]
 [0.68572236]
 [0.43057385]
 [0.12803385]
 [0.36686361]
 [0.24671721]
 [0.86357919]
 [0.52290546]
 [0.48418577]
 [0.77898762]
 [0.7527056 ]
 [0.82294716]
 [0.05592674]
 [0.16088572]
 [0.60792554]
 [0.76390685]
 [0.06456823]
 [0.21695574]
 [0.07080704]
 [0.86197101]
 [0.73382996]
 [0.76135915]
 [0.51397814]
 [0.66718528]
 [0.20844474]
 [0.02871127]
 [0.1727203 ]
 [0.40211045]
 [0.72228113]
 [0.21

In [ ]:
# Compute the RMSE between the original matrix R and the reconstructed matrix R_reconstructed

RMSE_missing = np.sqrt(np.mean((R_f - R_reconstructed_missing)**2))

print("RMSE_missing:", RMSE_missing)

# Compute the RMSE of U and V

RMSE_U_missing = np.sqrt(np.mean((u - U_factorized)**2))

RMSE_V_missing = np.sqrt(np.mean((v - V_factorized)**2))

print("RMSE_U_missing:", RMSE_U_missing)
print("RMSE_V_missing:", RMSE_V_missing)

RMSE_missing: 2.1188915087976775e-15
RMSE_U_missing: 0.016151981573954834
RMSE_V_missing: 0.016973077671913125


In [ ]:
# Discuss other missing proportions for data generation and discuss their impacts in the reconstruction process

# missing protions from 0.1 to 0.9

missing_proportions = [0.1*i for i in range(1,10)]

for missing_proportion in missing_proportions:

    num_missing = int(missing_proportion * total_elements)

    missing_indices = np.unravel_index(
        np.random.choice(total_elements, num_missing, replace=False), (N, M)
    )

    R_missing = np.copy(R_f)
    
    R_missing[missing_indices] = np.nan

    U_factorized,V_factorized = SGD_factorization(0.01, 0,1000, R_missing,False,1)

    R_reconstructed_missing = U_factorized @ V_factorized.T

    RMSE_missing = np.sqrt(np.mean((R_f - R_reconstructed_missing)**2))

    print("******* Missing Proportion:", round(missing_proportion,1),"********")
    print("RMSE_missing:", RMSE_missing)

    RMSE_U_missing = np.sqrt(np.mean((u - U_factorized)**2))

    RMSE_V_missing = np.sqrt(np.mean((v - V_factorized)**2))

    print("RMSE_U_missing:", RMSE_U_missing)

    print("RMSE_V_missing:", RMSE_V_missing)



******* Missing Proportion: 0.1 ********
RMSE_missing: 2.4052637664011314e-15
RMSE_U_missing: 0.002903838579411324
RMSE_V_missing: 0.002980951874122636
******* Missing Proportion: 0.2 ********
RMSE_missing: 2.9979264203027447e-15
RMSE_U_missing: 0.02272930551813848
RMSE_V_missing: 0.02416856307586616
******* Missing Proportion: 0.3 ********
RMSE_missing: 2.0339542859740246e-15
RMSE_U_missing: 0.019673670629673425
RMSE_V_missing: 0.020804598065549313
******* Missing Proportion: 0.4 ********
RMSE_missing: 2.5091924606756215e-15
RMSE_U_missing: 0.032227756013432245
RMSE_V_missing: 0.034866752088528304
******* Missing Proportion: 0.5 ********
RMSE_missing: 2.0882499147952527e-15
RMSE_U_missing: 0.008849151535654218
RMSE_V_missing: 0.009179326085041645
******* Missing Proportion: 0.6 ********
RMSE_missing: 5.634315082766128e-13
RMSE_U_missing: 0.004462408611580909
RMSE_V_missing: 0.004593396604888851
******* Missing Proportion: 0.7 ********
RMSE_missing: 3.241459235943753e-08
RMSE_U_missing

**Discussion**: 

**Reconstructed Matrix**:  
1. For the low and moderate missing value proportion such as 10% - 50%, the RMSE is very low, indicating that the factorization can approximate the matrix accuractely. 
2. For the moderate and high missing proportion such as 60% - 80%, the RMSE increases as the missing proportion increases, showing that a slight degradation in the matrix reconstruction quality. And the degradation becomes notable as the missing proportion reaches 80%.It shows that matrix factorization methods are resilient to a moderate amount of missing data.
3. For the very high missing proportion such as 90%, the RMSE rises significantly. The factorization struggles to approximate the original matrix since the data in the matrix becomes pretty sparse, and there is insufficient data to capture the underlying structure effectively.

**Reconstructed U and V**:
1. For the low missing proportions such as 10% - 30%, the RMSEs for U and V are very low at these missing proportions, indicating that the latent factors U and V are accurately estimated.  
2. For the moderate missing proportions such as 40% - 60%, the RMSEs remains very low too at these missing proportions, indicating that the latent factors U and V can be accurately estimated.    
3. For the high missing proportions such as 70% - 90%, the RMSEs for U and V becomes relatively high. It shows that for the sparse matrix, the latent factors are difficult to be estimated because the information becomes less and less.

In [ ]:
# Generate Matrix with Rank 10

N,M = 100,50

full_rank_matrix = np.random.rand(N, M)
    
# Perform SVD decomposition
U_10, s_10, Vt_10 = np.linalg.svd(full_rank_matrix, full_matrices=False)

# Keep only the top `rank` singular values
s_10[10:] = 0
    
# Reconstruct the matrix

R_10 = U_10 @ np.diag(s_10) @ Vt_10

# choose a moderate missing data proportions
num_missing = int(0.3 * total_elements)

missing_indices = np.unravel_index(
    np.random.choice(total_elements, num_missing, replace=False), (N, M)
)

R_missing_10 = np.copy(R_10)

R_missing_10[missing_indices] = np.nan

# decompose the matrix using different number of singular components

singular_components = [1,2,3,4,5,6,7,8,9,10, 20, 30, 40, 50]

for K in singular_components:

    U_reconstructed, s_constructed, V_reconstructed = np.linalg.svd(R_10, full_matrices=False)
    
    s_constructed[K:] = 0

    R_reconstructed_k = U_reconstructed @ np.diag(s_constructed) @ V_reconstructed

    RMSE_missing = np.sqrt(np.mean((R_10 - R_reconstructed_k)**2))

    print("******* Singular Number:", K ,"********")
    print("RMSE_reconstruction:", RMSE_missing)
    

******* Singular Number: 1 ********
RMSE_reconstruction: 0.1785662344904074
******* Singular Number: 2 ********
RMSE_reconstruction: 0.16612216880883932
******* Singular Number: 3 ********
RMSE_reconstruction: 0.1537641110171347
******* Singular Number: 4 ********
RMSE_reconstruction: 0.1405963076679247
******* Singular Number: 5 ********
RMSE_reconstruction: 0.12698460343689363
******* Singular Number: 6 ********
RMSE_reconstruction: 0.112475592395448
******* Singular Number: 7 ********
RMSE_reconstruction: 0.09641838014146753
******* Singular Number: 8 ********
RMSE_reconstruction: 0.07733966356176836
******* Singular Number: 9 ********
RMSE_reconstruction: 0.05442717199982621
******* Singular Number: 10 ********
RMSE_reconstruction: 2.460319285765109e-15
******* Singular Number: 20 ********
RMSE_reconstruction: 2.4639862668001584e-15
******* Singular Number: 30 ********
RMSE_reconstruction: 2.4676066743274875e-15
******* Singular Number: 40 ********
RMSE_reconstruction: 2.4701214049

**Discussion**:   
I use the a rank-10 matrix with 30% missing values.
1. For the number of singular value smaller than or equal to the Rank of the matrix. As the number of singular value increases, the reconstruction effect increases because we use more features. And the MSE becomes very small as the number of singualr value is equal to the Rank of the matrix.
2. For the number of singular values bigger than the Rank of the matrix. As the number of singular value increases, the reconstruction effect remains stable because singular values after the 10th are quite small so there is very little information in those singular components of the matrix.